In [1]:
# diferencia entre groupByKey y reduceByKey (mejor pues trabaja bajo demanda sin cargar todos los datos.)
# en un claster es mejor usar el segundo que el primero.

ls = sc.textFile('data/coupon150720.csv')

In [2]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

## Top ten de aerolineas por ingresos.

In [6]:
def getAlIngreso(l):
    elems = l.split(',')
    al = elems[4]
    ingreso = float(elems[6])
    return(al, ingreso)

In [7]:
getAlIngreso(ls.first())

(u'9W', 56.79)

In [9]:
als = ls.map(getAlIngreso)

In [10]:
als.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [11]:
# Agrupamos
als.groupByKey().take(2)

[(u'', <pyspark.resultiterable.ResultIterable at 0x7f1f7405aa90>),
 (u'BE', <pyspark.resultiterable.ResultIterable at 0x7f1f7405ac50>)]

In [13]:
# sumar los números para hallar los ingresos totales. Deberíamos controlar los outlayers.
totales = als.groupByKey().mapValues(lambda vs: sum(vs))

In [14]:
# si vamos a reutilizar mucho un elemento, ej. totales debemos usar totales.cache() 
# para guardarlo para no repetir operaciones. Si no repite todo lo hecho anteriormente.

In [15]:
# Ordemanos por la suma de valores. Sacamos los top ten
totales.sortBy(lambda x: x[1], False).take(10)

[(u'GA', 20712226.56999984),
 (u'BA', 12443986.980000038),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.900000253),
 (u'LH', 7715608.339999713),
 (u'QR', 6935584.369999946),
 (u'SV', 6286299.340001837),
 (u'UA', 5151815.289999981),
 (u'JJ', 5066300.509999913),
 (u'AA', 4151996.4599999674)]

In [22]:
# Repetimos lo mismo pero en vez de groupByKey por reduceByKey. CASI SIEMPRE USAR MEJOR REDUCEBYKEY!!
totales = als.reduceByKey(lambda e,acum: e+acum)

In [21]:
# Lo mismo de arriba, pero añadiendo ya la parte de sort.
totales = als.reduceByKey(lambda e,acum: e+acum).sortBy(lambda x: x[1], False).take(10)

[(u'GA', 20712226.569999855),
 (u'BA', 12443986.980000002),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.9000001205),
 (u'LH', 7715608.339999718),
 (u'QR', 6935584.369999997),
 (u'SV', 6286299.340001843),
 (u'UA', 5151815.29),
 (u'JJ', 5066300.509999905),
 (u'AA', 4151996.4599999613)]

## Top cinco ingreso medio por pasajero.

In [29]:
als.groupByKey().mapValues(lambda vs: sum(vs)/len(vs))\
.sortBy(lambda x: x[1], False).take(5)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.673986486487),
 (u'GA', 991.2527671691715),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035)]

### Lo mismo pero con reduceByKey

In [32]:
# paso necesario para crear una tupla.
als.map(lambda x: (x[0], (x[1], 1))).take(2)

[(u'9W', (56.79, 1)), (u'9W', (84.34, 1))]

In [33]:
def calculaMediaReduce(e, acum):
    sumaTotal = acum[0]
    numElems = acum[1]
    return(e[0]+sumaTotal, e[1] + numElems)

In [34]:
als.map(lambda x: (x[0], (x[1], 1))).reduceByKey(calculaMediaReduce).take(2)
# devuelve: aerolinea, precio medio, cantidad tickets

[(u'', (478.31000000000006, 18811)), (u'BE', (64073.32999999998, 788))]

In [35]:
# calculamos la media para cada grupo
tr = als.map(lambda x: (x[0], (x[1], 1))).reduceByKey(calculaMediaReduce)

In [36]:
tr.mapValues(lambda x: x[0]/x[1]).sortBy(lambda x: x[1], False).take(5)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.6739864864867),
 (u'GA', 991.2527671691723),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035)]